In [1]:
import requests 
from bs4 import BeautifulSoup
from tqdm import tqdm
  
URL = "https://www.fairprice.com.sg/categories" 
r = requests.get(URL)

soup = BeautifulSoup(r.content, 'html.parser')
div_elements = soup.find_all('a', attrs={'data-testid':"sub-category-item"})

cats = []
for div in tqdm(div_elements):
    cats.append(div.get('href')[10:])

print(f'Number of sub-categories in fairprice: {len(cats)}')

100%|█████████████████████████████████████| 146/146 [00:00<00:00, 598016.00it/s]

Number of sub-categories in fairprice: 146


In [5]:
all_data = {} 
error_cat = [] #to catch errors

In [6]:
import math

# Loop through each category
for cat in cats: 
    collect = []
    
    BASE_URL = 'https://www.fairprice.com.sg/category/'
    print(f'Now doing sub-category: {cat}')
    
    try:
        page = requests.get(BASE_URL + cat)
        # Attempt to get item count of each sub-cat of every cat; will not work for sub-cat with sub-cats
        soup = BeautifulSoup(page.content, 'html.parser')
        
        pages = 50 # Max that can be got; luckily most cats do not exceed this, only 2 or 3 do
        items = soup.find_all('div', class_='jBLMxC')
    
        if len(items[0].find_all('span'))>1:
            num_items = int(items[0].find_all('span')[1].get_text().split(' ')[0])
            pages = math.ceil(num_items/20)
            print(f'{num_items} found; {pages} pages to go through...')
        else:
            print('!!! CANNOT access number of items; likely layer 2 sub category, will extract max of 50 pages.')


        for i in tqdm(range(1, pages+1)):
            url = f'https://website-api.omni.fairprice.com.sg/api/layout/category/v2?algopers=prm-ppb-1%2Cprm-ep-1%2Ct-epds-1%2Ct-ppb-0%2Ct-ep-0&category={cat}&experiments=searchVariant-B%2CtimerVariant-Z%2CinlineBanner-A%2CsubstitutionBSVariant-B%2Cgv-A%2Cshelflife-B%2Cds-A%2CSDND_delivery_reason-A%2Cls_comsl-B%2Ccompatt-B%2Cls_deltime-ogA%2Cls_deltime-feA%2Ccartfiller-a%2Ccatnav-hide%2Ccatbubog-B%2Csbanner-A%2Ccount-b%2Ccam-b%2Cpriceperpiece-b%2Cls_deltime-sortA%2Cpromobanner-c%2Calgopers-b%2Cdelivery_pref_ffs-A%2Cdelivery_pref_pfc-B%2Ccrtalc-B&includeTagDetails=true&orderType=DELIVERY&page={i}&url={cat}'
            response = requests.get(url)
            
            # Retrieve json data
            if response.status_code == 200:
                json_data = response.json()
                try: # List index out of bound for ~60 of the sub-categories; replace with index 1
                    x = json_data['data']['page']['layouts'][2]['value']['collection']['product']
                    collect += x
                except:
                    # To catch the end for categories where page limit is unknown
                    print(f'page {i} does not exist.')
                    break
            else: 
                print(f'error at getting json at {i}')

        all_data[cat] = collect
        print(f'Done with sub-category: {cat}; Total of {len(collect)} items found')
        
    except Exception as e:
        print(f'SOME ERROR OCCURED FOR sub-category {cat}; REDO')
        print(e)
        error_cat.append(cat)

Now doing sub-category: coles
20 found; 1 pages to go through...


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.34it/s]


Done with sub-category: coles; Total of 20 items found
Now doing sub-category: morrisons
32 found; 2 pages to go through...


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  7.99it/s]


Done with sub-category: morrisons; Total of 32 items found
Now doing sub-category: marks--spencer
621 found; 32 pages to go through...


...

  0%|                                                    | 0/52 [00:00<?, ?it/s]

page 1 does not exist.
Done with sub-category: gardening-3; Total of 0 items found
Now doing sub-category: travel-essentials-1


181 found; 10 pages to go through...


  0%|                                                    | 0/10 [00:00<?, ?it/s]

page 1 does not exist.
Done with sub-category: travel-essentials-1; Total of 0 items found


In [36]:
fails = []
for k, v in all_data.items():
    if len(v) == 0:
        fails.append(k)
fails

['diapers-3074457345616687146-1', 'ready-to-eat--1']

In [19]:
all_data_backup = all_data

In [35]:
# Exact same as above, just replace with index 1; 

for cat in fails:
    collect = []
    
    BASE_URL = 'https://www.fairprice.com.sg/category/'
    print(f'Now doing sub-category: {cat}')
    
    try:
        page = requests.get(BASE_URL + cat)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        pages = 50
        
        items = soup.find_all('div', class_='jBLMxC')
    
        if len(items[0].find_all('span'))>1:
            num_items = int(items[0].find_all('span')[1].get_text().split(' ')[0])
            pages = math.ceil(num_items/20)
            print(f'{num_items} found; {pages} pages to go through...')
        else:
            print('!!! CANNOT access number of items; likely layer 2 sub category, will extract max of 50 pages.')


        for i in tqdm(range(1, pages+1)):
            url = f'https://website-api.omni.fairprice.com.sg/api/layout/category/v2?algopers=prm-ppb-1%2Cprm-ep-1%2Ct-epds-1%2Ct-ppb-0%2Ct-ep-0&category={cat}&experiments=searchVariant-B%2CtimerVariant-Z%2CinlineBanner-A%2CsubstitutionBSVariant-B%2Cgv-A%2Cshelflife-B%2Cds-A%2CSDND_delivery_reason-A%2Cls_comsl-B%2Ccompatt-B%2Cls_deltime-ogA%2Cls_deltime-feA%2Ccartfiller-a%2Ccatnav-hide%2Ccatbubog-B%2Csbanner-A%2Ccount-b%2Ccam-b%2Cpriceperpiece-b%2Cls_deltime-sortA%2Cpromobanner-c%2Calgopers-b%2Cdelivery_pref_ffs-A%2Cdelivery_pref_pfc-B%2Ccrtalc-B&includeTagDetails=true&orderType=DELIVERY&page={i}&url={cat}'
            response = requests.get(url)
            if response.status_code == 200:
                json_data = response.json()
                try:
                    x = json_data['data']['page']['layouts'][1]['value']['collection']['product']
                    collect+=x
                except:
                    print(f'page {i} does not exist.')
                    break
            else: 
                print(f'error at getting json at {i}')

        all_data[cat] = collect
        print(f'Done with sub-category: {cat}; Total of {len(collect)} items found')
    except Exception as e:
        print(f'SOME ERROR OCCURED FOR sub-category {cat}; REDO')
        print(e)
        error_cat.append(cat) 

Now doing sub-category: mandarin-oranges-1
31 found; 2 pages to go through...


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.95it/s]


Done with sub-category: mandarin-oranges-1; Total of 31 items found
Now doing sub-category: plants-and-decorations
176 found; 9 pages to go through...


100%|█████████████████████████████████████████████| 9/9 [00:06<00:00,  1.50it/s]


Done with sub-category: plants-and-decorations; Total of 176 items found
Now doing sub-category: infant-formula--1
53 found; 3 pages to go through...


...

 96%|█████████████████████████████████████████▎ | 50/52 [00:41<00:01,  1.21it/s]

page 51 does not exist.
Done with sub-category: gardening-3; Total of 1000 items found
Now doing sub-category: travel-essentials-1


181 found; 10 pages to go through...


100%|███████████████████████████████████████████| 10/10 [00:05<00:00,  1.78it/s]

Done with sub-category: travel-essentials-1; Total of 181 items found


In [37]:
import json

file_path = "output_final.json"
with open(file_path, 'w') as json_file:
    json.dump(all_data, json_file)